<a href="https://colab.research.google.com/github/Tiger-A/Yandex_Data-Science/blob/master/ml_for_texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from google.colab import drive


import torch
# import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    train_test_split
    )


import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:

try:
    '''путь loc'''
    drive.mount('/content/drive')
    df_tweets = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ds_yandex/db/toxic_comments.csv')

except:
    '''путь для ревьюера'''
    df_tweets = pd.read_csv('/datasets/toxic_comments.csv')

Mounted at /content/drive


In [10]:
#  Для тестов возьмем 500
df_tweets = df_tweets.sample(500).reset_index(drop=True)

In [ ]:
# Напишите функцию clear_text(text), которая оставит в тексте только кириллические символы и пробелы. На вход она принимает текст, а возвращает очищенный текст. Дополнительно уберите лишние пробелы.
# Напечатайте на экране исходный текст, а затем очищенный и лемматизированный тексты (уже в прекоде).

# import pandas as pd
# from pymystem3 import Mystem
# import re

# data = pd.read_csv('/datasets/tweets.csv')
# corpus = list(data['text'])


# def lemmatize(text):
#     m = Mystem()
#     lemm_list = m.lemmatize(text)
#     lemm_text = "".join(lemm_list)

#     return lemm_text


# def clear_text(text):
#     text = re.sub(r'[^а-яА-ЯёЁ ]', ' ', text)
# #    print(text)

#     return " ".join(text.split())


In [11]:

# !pip install pandarallel

# from pandarallel import pandarallel
# from tqdm import tqdm

# tqdm.pandas(desc="progress")
# pandarallel.initialize(progress_bar = True)

# nlp = spacy.load("en_core_web_sm")
# data['lemmatized_text'] = data['text'].parallel_apply(lambda x, nlp=nlp: " ".join([token.lemma_ for token in nlp(x)]))

In [12]:
features = df_tweets.drop(['toxic'], axis=1)
target = df_tweets['toxic']

In [ ]:
from transformers import BertModel, BertTokenizer

# Загрузка предобученного токенизатора
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

# Загрузка предобученной модели
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')


In [26]:



# построчно Преобразуем текст в номера токенов из словаря методом encode() (англ. «закодировать»):

tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x[:512], add_special_tokens=True))

#  Вычислим макс длину токена
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# Применим метод padding (англ. «отступ»), чтобы после токенизации длины исходных текстов в корпусе были равными. Только при таком условии будет работать модель BERT.
# Пусть стандартной длиной вектора n будет длина наибольшего во всём датасете вектора. Остальные векторы дополним нулями:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
# Теперь поясним модели, что нули не несут значимой информации. Это нужно для компоненты модели, которая называется «внимание» (англ. attention).
# Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения:
attention_mask = np.where(padded != 0, 1, 0)

In [15]:
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

Паша, [6/16/2023 6:43 AM]
И еще, как я понял, важно прописать
`tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x[:512], add_special_tokens=True))`

или
`tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x,
                               add_special_tokens=True,
                               max_length=512,
                               truncation=True))`

In [16]:
attention_mask.shape


(1000, 3296)

In [27]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

In [24]:
# # #  преобразование текстов в эмбеддинги
# # batch_size = 100
# # embeddings = []
# # for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
# #         batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
# #         attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

# #         with torch.no_grad():
# #             batch_embeddings = model(batch, attention_mask=attention_mask_batch)

# #         embeddings.append(batch_embeddings[0][:,0,:].numpy())




# # batch_size = 100
# # embeddings = []

# # for i in notebook.tqdm(range(padded.shape[0] // batch_size)):

# #     batch = torch.LongTensor(padded[batch_size * i:batch_size * (i + 1)]).unsqueeze(1)  # добавляем размерность пакета
# #     attention_mask_batch = torch.LongTensor(attention_mask[batch_size * i:batch_size * (i + 1)]).unsqueeze(1)  # добавляем размерность пакета

# #     with torch.no_grad():
# #         batch_embeddings, _ = model(batch, attention_mask=attention_mask_batch)


# #     embeddings.append(batch_embeddings[0][:, 0, :].numpy())

# # # Объединяем эмбеддинги в один массив
# # embeddings = np.concatenate(embeddings, axis=0)


# for i in notebook.tqdm(range((padded.shape[0] + batch_size - 1) // batch_size)):
#     start = batch_size * i
#     end = min(batch_size * (i + 1), padded.shape[0])
#     batch = torch.LongTensor(padded[start:end]).unsqueeze(1)
#     attention_mask_batch = torch.LongTensor(attention_mask[start:end]).unsqueeze(1)

#     # with torch.no_grad():
#     #     batch_embeddings = model(batch, attention_mask=attention_mask_batch)[0]
#     with torch.no_grad():
#         batch_embeddings = model(batch, attention_mask=attention_mask_batch)
#     embeddings.append(batch_embeddings[:, 0, :].numpy())

# # Объединяем эмбеддинги в один массив
# embeddings = np.concatenate(embeddings, axis=0)


  0%|          | 0/10 [00:00<?, ?it/s]

ValueError: ignored

In [28]:
features = np.concatenate(embeddings)


# обучите и протестируйте модель
# < напишите код здесь >
train, test = train_test_split(df_tweets, shuffle=False, test_size=0.5)

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.5, random_state=12345
)

print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

(500, 768)
(500, 768)
(500,)
(500,)


## Обучение

In [29]:
model = LogisticRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
acc = accuracy_score(y_test, pred)

In [31]:
acc

0.914

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны